# Converitdor de archivos excel a parquet (Formato binario)

In [ ]:
# Importar las librerias
import polars as pl
import pandas as pd
import os
import gc

Como primera instacia, declaramos variable global donde se encuentra nuesta informacion para asi recorerrla en los diferentes archivos

In [ ]:
# Comando para poder ubicar nuestra ruta actual del proyecto, util en grandes proyecto para poder manejar rutas relativas 
path_project = os.getcwd()
print(path_project)

Como primer paso, ubicaremos la carpata "Archivo"

In [ ]:
path_archivo = os.path.join(path_project, '..', 'data', 'raw', 'Archivo 3')
print(path_archivo)

Para confirmar que ya podemos acceder a nuestros archivos, ahora se listaran

In [ ]:
print(os.listdir(path_archivo))

Ahora pasamos a procesar los archivos a parquet, pero para ellos primero debemos de entender primero.

Un archivo Parquet es un formato de almacenamiento de datos columnares desarrollado por Apache. Está diseñado para manejar grandes volúmenes de información de manera eficiente, permitiendo compresión, lectura selectiva de columnas y compatibilidad con múltiples lenguajes y frameworks (Python, Spark, Hadoop, etc.). A diferencia de formatos como CSV o Excel, que almacenan datos fila por fila, Parquet organiza los datos por columnas, lo que lo hace ideal para procesamiento analítico y Big Data.

| Característica           | CSV                         | Excel (XLSX)              | Parquet                                           |
| ------------------------ | --------------------------- | ------------------------- | ------------------------------------------------- |
| **Estructura**           | Fila por fila (row-based)   | Fila por fila (row-based) | Columna por columna (column-based)                |
| **Compresión**           | Poca / manual               | Moderada                  | Alta (automática y optimizada)                    |
| **Velocidad de lectura** | Lenta con grandes datos     | Lenta                     | Muy rápida (lectura selectiva)                    |
| **Soporte de tipos**     | Texto y números básicos     | Texto, números, fechas    | Tipos complejos (listas, fechas, decimales, etc.) |
| **Tamaño de archivo**    | Grande                      | Grande                    | Reducido (por compresión)                         |
| **Uso típico**           | Intercambio simple de datos | Reportes y ofimática      | Big Data, análisis, machine learning              |



In [ ]:
def es_numero(valor: str) -> bool:
    """
        Funcion que nos permite identificar si un valor es numerico o no
        Args:
            valor (str): valor a evaluar.
        Returns:
            bool: True si el valor es numerico, False en caso contrario.
    """
    try:
        float(valor)
        return True
    except ValueError:
        return False

In [ ]:
# Converit a parquet los archivos de excel que no son parquet
for file in [file for file in os.listdir(path_archivo) if not file.endswith('.parquet') and not file.startswith('ClasificacionRiesgoCard')]: 
    print(file)
    df = pd.read_excel(os.path.join(path_archivo, file), sheet_name='BASE', header=4, dtype=object)
    df = df.convert_dtypes()
    df = df.rename(columns={
        col: str(col).strip().lower().replace(' ', '_').replace('-', '_') if not es_numero(str(col)) else f'col_{col}' for col in df.columns
    })

    for key, value in {col: df[col].dtype for col in df.columns}.items():
        try:
            if "date" in value: df[key] = pd.to_datetime(df[key], errors="coerce")
            elif "int" in value or "float" in value: df[key] = pd.to_numeric(df[key], errors="coerce")
            else: df[key] = df[key].astype("string").str.strip()
        except Exception as e: df[key] = df[key].astype("string").str.strip()

    df.to_parquet(os.path.join(path_archivo, f'{file.replace('.xlsx', '')}.parquet'), engine="pyarrow")
    
    del df
    gc.collect()

En Python, gc es el módulo que controla el Garbage Collector (recolector de basura).
El Garbage Collector es el mecanismo que administra la memoria automáticamente, detectando objetos que ya no se usan y liberando el espacio que ocupaban en RAM.

Muy bien, una vez generado los archivos podemos validar si quedaron de forma correcta

In [ ]:
df = pd.read_parquet(os.path.join(path_archivo, 'Reporte Cronicos ESE Oriente JUNIO 2018.parquet'))
print(df)